# Fallstudie Flink
## Standortentscheidung Erweiterung 

## Modell

### Indexmengen
$s \in S$ : Menge der potenziellen DarkStores (Standorte)

$i \in I$ : Menge der (Nachfrage) i-Koordinaten 

$j \in J$ : Menge der (Nachfrage) j-Koordinaten  



### Parameter
$n_{ij}$ : Nachfrage an $i$$j$

$u_{s}$ : Lagerumschlagsleistung an Standort $s$

$c_{s}$ : Errichtungskosten für Standort $s$

$ki_{s}$ : $i$-Kooridnate von Standort $s$ 

$kj_{s}$ : $j$-Kooridnate von Standort $s$ 



### Entscheidungsvariablen

$V_{sij} \in \{0,1\}$ : Binäre Standort-Versorgungsvariable

$Y_{s} \in \{0,1\}$ : Binäre Standortausbauvariable

### ----------
#### Erweiterung 

$NA_{sij} \ge 0$ : Nachfrageabdeckung von Standort $S$ an Quadrant  $ij$ 

$Q_{ij} \in \{0,1\}$ : Binäre Versorgungsvariable (Quadrant)



### Alte Zielfunktion

Maximiere Nachfrageabdeckung:

Max $NA = \sum_{s,i,j} (V_{s,i,j} * n_{i,j})$   


### Neue Zielfunktion

Maximiere Nachfrageabdeckung erweiterung:

Max $NA = \sum_{s,i,j} (NA_{s,i,j})$  

### Nebenbedingungen

**(1) Budget einhalten**

$\sum_{s} (c_s*Y_s) \le 1.000.000 $

Summe über: 

Kosten  für Ausbau * Entscheidung Ausbau <= Budget

**(2) Lieferzeit einhalten**

$(|ki_s - i| + |kj_s - j|)* v_{s,i,j} \le 5 $

$∀ s,i,j$


Prüfe für jedes $s, i , j$:

Entfernung  <= 5, wenn $ij$ von $s$ beliefert wird

**(3) Höchstens Doppelbedienung der Quandranden**

$\sum_s v_{s,i,j} \le 2$

$∀ i, j$

Prüfe für alle Koordinaten, ob Belieferung kleiner gleich 1.

**(4) Kapazitäten einhalten**

$\sum_{i,j} (NA_{s,i,j}) \le u_s *Y_s $ 

$∀ s$

Prüfe für jeden Standort:

Summe Versorgung (ja/nein) * Nachfrageabdeckung  ist kleiner (gleich) als Umschlagsleistung. 

**(5) Nachfragebedienung**

$\sum_s NA_{s,i,j} = n_{i,j} *Q_{i,j}$ 

$∀ i,j$

Summe aus der Nachfrageabdeckung aller Standorte für jede i und j ergibt die Nachfrge in i j, wenn der Quadrant bedient wird

**(6) Nur wenn Ausbau, darf NA über 0 sein**

$ NA_{s,i,j} <= n_{i,j} * Y_s$ 

$∀ s,i,j$

**(7) Nachfrageabdeckung nur über 0 wenn Versorgungsvariable 1**

$NA_{s,i,j} <= n_{i,j} * V_{s,i,j}$ 

$∀ s,i,j$



## Implementierung

## Datenaufbereitung


1.   Fallstudien-Daten in Google-Drive laden
2.   Google-Drive mit Colab-Notebook verbinden
3.   Daten mit `pandas` laden



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Ordner finden
! ls drive/MyDrive/Industrielles_Management/Daten/Fallstudie

Nachfrage.csv  Standorte.csv


In [5]:
# Pfad zurückgeben
! cd drive/MyDrive/Industrielles_Management/Daten/Fallstudie && pwd

/content/drive/MyDrive/Industrielles_Management/Daten/Fallstudie


In [6]:
# Daten laden
import pandas as pd

In [7]:
path = "/content/drive/MyDrive/Industrielles_Management/Daten/Fallstudie"

In [8]:
# Nachfragedaten lesen & speichern
nachfrage_df = pd.read_csv(f"{path}/Nachfrage.csv", sep=";")

In [9]:
# Nachfragedaten ausgeben
nachfrage_df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,7,91,34,77,33,47,43,51,81,28,88,74,81
1,1,73,82,76,51,80,42,83,19,37,37,85,18,67
2,2,85,57,67,14,1,92,41,32,16,83,65,39,18
3,3,42,87,23,56,15,99,60,85,90,68,9,63,88
4,4,23,93,36,65,96,31,8,54,27,74,59,70,36
5,5,2,71,34,38,62,45,86,95,93,12,83,59,17
6,6,31,41,54,1,97,73,5,73,44,93,94,2,56
7,7,61,44,86,1,69,65,32,95,61,39,7,61,31
8,8,59,49,1,70,86,26,62,40,96,82,3,20,83
9,9,10,38,37,77,16,94,34,67,96,64,32,30,63


In [10]:

# nachfrage is a dictionary of nachfrage_df. The first key are the values of column 1 (i coordinate) 
# the second key are the values of the columns (j coordinate)
# skip the first column for the second key since it's not an j cooridnate

nachfrage = {int(row[0]): {int(col): value for col, value in row.items() if col != 'Unnamed: 0'} for _, row in nachfrage_df.iterrows()}

# test if it worked

# nachfrage i=0 & j=7
print(nachfrage[0][7])

#nachfrage i=10 & j=12
print(nachfrage[10][12])

print(nachfrage[12][12])


51
43
83


In [11]:
# Standortdaten lesen & speichern
standorte = pd.read_csv(f"{path}/Standorte.csv", sep=";", decimal=",")

In [12]:
# Standortdaten ausgeben
standorte

,Potenzielle_Standorte,i_Koordinate,j_Koordinate,Lagerumschlagleistung,Errichtungskosten
0,0,1,0,500,100000
1,1,8,11,600,90000
2,2,10,4,500,95000
3,3,10,10,750,101000
4,4,13,6,725,98500
5,5,4,8,750,97500
6,6,13,7,455,105000
7,7,4,6,3000,225000
8,8,12,6,1400,300000
9,9,4,4,2250,200000


## Indexmengen

In [13]:
# Standorte
S = standorte["Potenzielle_Standorte"].unique().tolist() # Menge der Standorte


In [14]:
# Ausgabe Standorte
S

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [15]:
# I Kooritnaten
I = [key for key, value in nachfrage.items()]

print('I-Koordinaten: ' + str(I))

I-Koordinaten: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [16]:
# J Koordinaten
J = [list(value.keys()) for value in nachfrage.values()][0]

print('J-Koordinaten: ' + str(J))


J-Koordinaten: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


## Parameter

In [17]:
# Standortkoordinaten

# I-Koordinate von Standort S
ki = standorte.set_index(["Potenzielle_Standorte"]).to_dict("dict")["i_Koordinate"]

# J-Koordinate von Standort S
kj = standorte.set_index(["Potenzielle_Standorte"]).to_dict("dict")["j_Koordinate"]

# check
print(ki)

print(kj)

{0: 1, 1: 8, 2: 10, 3: 10, 4: 13, 5: 4, 6: 13, 7: 4, 8: 12, 9: 4}
{0: 0, 1: 11, 2: 4, 3: 10, 4: 6, 5: 8, 6: 7, 7: 6, 8: 6, 9: 4}


In [18]:
# Umschlagsleistung
u = standorte.set_index(["Potenzielle_Standorte"]).to_dict("dict")["Lagerumschlagleistung"]
#check
u


{0: 500,
 1: 600,
 2: 500,
 3: 750,
 4: 725,
 5: 750,
 6: 455,
 7: 3000,
 8: 1400,
 9: 2250}

In [19]:
# Errichtungskosten
c = standorte.set_index(["Potenzielle_Standorte"]).to_dict("dict")["Errichtungskosten"]
#check
c

{0: 100000,
 1: 90000,
 2: 95000,
 3: 101000,
 4: 98500,
 5: 97500,
 6: 105000,
 7: 225000,
 8: 300000,
 9: 200000}

In [20]:
# Nachfrage
n = nachfrage
#check
n[0][0]

7

## Solver

In [21]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 19.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.
tensorflow-metadata 1.12.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.12 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.


In [22]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')
# Erstelle einen Solver
#solver = pywraplp.Solver('solver_fallstudie', pywraplp.Solver.GLPK_MIXED_INTEGER_PROGRAMMING)


## Entscheidungsvariablen

In [24]:
# Entscheidung über Versorgung von Quandrand ij

Q={}
for i in I:
  for j in J:
        Q[i,j] = solver.BoolVar(f"{i},{j}")

#check
Q[1,0]


1,0

In [25]:
# Binäre Versorgungsvariable
V={}
for s in S: 
  for i in I:
    for j in J:
        V[s,i,j] = solver.BoolVar(f"{s},{i},{j}")

#check
V[1,3,7]

1,3,7

In [26]:
# Binäre Standortausbauvariable
Y={}
for s in S:
  Y[s] = solver.BoolVar(f"{s}")


#check
Y[1]

1

In [27]:
# Nachfrageabdeckung 

#Obergrenze unendlich (float)
infinity = solver.infinity()

NA={}
for s in S:
  for i in I:
      for j in J:
        NA[s,i,j] = solver.NumVar(0.0, infinity, f"{s},{i},{j}")


In [28]:
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

Anzahl Entscheidungsvariablen = 3728


## Zielfunktion



### Alte Zielfunktion

Maximiere Nachfrageabdeckung:

Max $NA = \sum_{s,i,j} (V_{s,i,j} * n_{i,j})$   


### Neue Zielfunktion

Maximiere Nachfrageabdeckung erweiterung:

Max $NA = \sum_{s,i,j} (NA_{s,i,j})$  


In [29]:
solver.Maximize(sum(NA[s,i,j] for s in S for i in I for j in J))

## Nebenbedingungen

**(1) Budget einhalten**

$\sum_{s} (c_s*Y_s) \le 1.000.000 $

Summe über: 

Kosten  für Ausbau * Entscheidung Ausbau <= Budget

In [30]:

solver.Add(sum(c[s]*Y[s] for s in S)<= 1000000)
 

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f6bd01eb7e0> >

**(2) Lieferzeit einhalten**

$(|ki_s - i| + |kj_s - j|)* v_{s,i,j} \le 5 $

$∀ s,i,j$


Prüfe für jedes $s, i , j$:

Entfernung  <= 5, wenn $ij$ von $s$ beliefert wird

In [31]:
for s in S:
 for i in I:
    for j in J:
      solver.Add(((abs(ki[s]-i) +abs(kj[s]- j))*V[s,i,j])<=5)


**(3) Höchstens Doppelbedienung der Quandranden**

$\sum_s v_{s,i,j} \le 2$

$∀ i, j$

Prüfe für alle Koordinaten, ob Belieferung kleiner gleich 1.

In [32]:
for i in I:
  for j in J:
    solver.Add(sum(V[s,i,j] for s in S)<=2)

**(4) Kapazitäten einhalten**

$\sum_{i,j} (V_{s,i,j}* NA_{s,i,j}) \le u_s *Y_s $

$∀ s$

Prüfe für jeden Standort:

Summe Versorgung (ja/nein) * Nachfrage  ist kleiner (gleich) als Umschlagsleistung. 

In [34]:
for s in S:
  solver.Add(sum(NA[s,i,j] for i in I for j in J)<= (u[s]* Y[s]))



**(5) Nachfragebedienung**

$\sum_s NA_{s,i,j} = n_{i,j} *Q_{i,j}$ 

$∀ i,j$

Summe aus der Nachfrageabdeckung aller Standorte für jede i und j ergibt die Nachfrge in i j, wenn der Quadrant bedient wird

In [35]:
for i in I:
  for j in J:
    solver.Add(sum(NA[s,i,j] for s in S) == (n[i][j]*Q[i,j]))

**(6) Nur wenn Ausbau, darf NA über 0 sein**

$ NA_{s,i,j} <= n_{i,j} * Y_s$ 

$∀ s,i,j$

In [36]:

for s in S:
    for i in I:
        for j in J:
            solver.Add(NA[s,i,j] <= n[i][j] * Y[s])
      

**(7) Nachfrageabdeckung nur über 0 wenn Versorgungsvariable 1**

$NA_{s,i,j} <= n_{i,j} * V_{s,i,j}$ 

$∀ s,i,j$



In [39]:
for s in S:
  for i in I:
    for j in J:
      solver.Add(NA[s,i,j] <= (n[i][j]* V[s,i,j]))

## Berechnung Lösung

In [40]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
  print('LÖSUNG:')
  print('Zielfunktionswert (Nachfrageabdeckung) =', solver.Objective().Value())
else:
  print('Problem hat keine Lösung')

LÖSUNG:
Zielfunktionswert (Nachfrageabdeckung) = 7481.999999999999


In [50]:
cost = 0
na = 0

for s in S:
  na_s =0
  if Y[s].solution_value() ==1:
    print("Wir bauen " + f"Standort: {s}" + " aus")
    cost += c[s]
    for i in I:
      for j in J:
        na_s += NA[s,i,j].solution_value()
    print("Nachfrageabdeckung Standort: " +str(na_s))
    na += na_s



print("Nachfrageabdeckung gesamt: " + str(na))
    
print("Gesamtkosten: " +str(cost))




Wir bauen Standort: 1 aus
Nachfrageabdeckung Standort: 600.0
Wir bauen Standort: 2 aus
Nachfrageabdeckung Standort: 499.99999999999994
Wir bauen Standort: 3 aus
Nachfrageabdeckung Standort: 750.0
Wir bauen Standort: 4 aus
Nachfrageabdeckung Standort: 724.9999999999999
Wir bauen Standort: 5 aus
Nachfrageabdeckung Standort: 750.0
Wir bauen Standort: 7 aus
Nachfrageabdeckung Standort: 1941.9999999999875
Wir bauen Standort: 9 aus
Nachfrageabdeckung Standort: 2215.0000000000214
Nachfrageabdeckung gesamt: 7482.000000000009
Gesamtkosten: 907000


In [59]:
# welche Quadranten beliefern wir von welchen Standorten?

# NAchfrageabdeckung Standort
na= 0 

for i in I:
  for j in J:
    if Q[i,j].solution_value()== 1:
      print("Wir beliefern: " + str(i) + str(j))
      for s in S:
        if NA[s,i,j].solution_value()> 0:
        #  if i ==3 :
         #   if j==10:
             print("Wird beliefert von: " + str(s))
             # print("NAchfrageabdeckung: " + str(NA[s,i,j].solution_value()))


Wir beliefern: 03
Wird beliefert von: 9
Wir beliefern: 04
Wird beliefert von: 9
Wir beliefern: 05
Wird beliefert von: 7
Wir beliefern: 06
Wird beliefert von: 7
Wir beliefern: 07
Wird beliefert von: 7
Wir beliefern: 08
Wird beliefert von: 5
Wir beliefern: 09
Wird beliefert von: 5
Wir beliefern: 12
Wird beliefert von: 9
Wir beliefern: 13
Wird beliefert von: 9
Wir beliefern: 14
Wird beliefert von: 9
Wir beliefern: 15
Wird beliefert von: 7
Wir beliefern: 16
Wird beliefert von: 9
Wir beliefern: 17
Wird beliefert von: 7
Wir beliefern: 18
Wird beliefert von: 7
Wir beliefern: 19
Wird beliefert von: 5
Wir beliefern: 110
Wird beliefert von: 5
Wir beliefern: 21
Wird beliefert von: 9
Wir beliefern: 22
Wird beliefert von: 9
Wir beliefern: 23
Wird beliefert von: 7
Wir beliefern: 24
Wird beliefert von: 9
Wir beliefern: 25
Wird beliefert von: 9
Wir beliefern: 26
Wird beliefert von: 7
Wir beliefern: 27
Wird beliefert von: 7
Wir beliefern: 28
Wird beliefert von: 7
Wir beliefern: 29
Wird beliefert von: 7

In [64]:
# welche Quadranten beliefern wir?

# NAchfrageabdeckung Standort
na= 0 

for i in I:
  for j in J:
    if Q[i,j].solution_value()== 1:
     # print("Wir beliefern: " + str(i) + str(j))
      for s in S:
        if NA[s,i,j].solution_value()> 0:
          if i ==10 :
            if j==12:
              print("Nachfrage in Quadrand: " +str(i)+ str(j) + " beträgt: " +str(n[i][j]))
              print("Wird beliefert von: " + str(s))
              print("NAchfrageabdeckung: " + str(NA[s,i,j].solution_value()))

Nachfrage in Quadrand: 1012 beträgt: 43
Wird beliefert von: 1
NAchfrageabdeckung: 28.999999999999993
Nachfrage in Quadrand: 1012 beträgt: 43
Wird beliefert von: 3
NAchfrageabdeckung: 14.000000000000014
